In [1]:
from numpy import sin, cos, pi, array
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation

FFMpegWriter = animation.writers['ffmpeg']
metadata = dict(title='Plane', artist='Pete Florence',
        comment='Fly time')
writer = FFMpegWriter(fps=20, metadata=metadata,bitrate=500)


# constant velocity
v = 1

def derivs(state, t):

    dqdt = np.zeros_like(state)
    
    dqdt[0] = -v * sin(state[2]) # Move origin, not plane (invert v to switch back)
    
    dqdt[1] = -v * cos(state[2]) # Move origin, not plane (invert v to switch back)

    dqdt[2] = 0 # for now, but later will be u

    return dqdt

# create a time array from 0..100 sampled at 0.05 second steps
dt = 0.05
t = np.arange(0.0, 20, dt)

# initial positions
# x and y are position of origin.  We move the origin, not the plane, so that the plane stays in the same spot in the plot.
# psi is yaw, and psidot is yawdot of plane
x = 0.0
y = 0.0
psi = -10.0

rad = pi/180.0

# initial state
state = np.array([x, y, psi*rad])

# integrate your ODE using scipy.integrate.
y = integrate.odeint(derivs, state, t)

xworld = y[:,0]
yworld = y[:,1]

x_obj1 = xworld + 5
y_obj1 = yworld + 3

x_obj2 = xworld - 4
y_obj2 = yworld + 7

In [2]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-15, 15), ylim=(-5, 20))
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text

def animate(i):
    thisx = [0, xworld[i], x_obj1[i], x_obj2[i]]
    thisy = [0, yworld[i], y_obj1[i], y_obj2[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template%(i*dt))
    return line, time_text

ani = animation.FuncAnimation(fig, animate, np.arange(1, len(y)),
    interval=20, blit=False, init_func=init)
ani.save('plane.mp4', fps=20, bitrate=5000, codec="h264")
plt.show()

In [ ]:
import os
import io
import base64
from IPython.display import HTML

video = io.open('/Users/pflomacpro/GeometricPDEs/plane.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))